In [1]:
import mlflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

import os
import sys
sys.path.append('/home/code/mlflow')
import models.Preprocessing as pre
import models.utils as ut
import models.GRU as gru

In [2]:
data_path = '/home/code/data/processed/week_dataset.csv'
df = pd.read_csv(data_path)

In [3]:
# 前処理用のパラメータ設定
week_1 = 7
week_2 = 14
week_3 = 21

weekly_mean_columns = ['Week Shipment']
weekly_total_columns = ['Status']
shift_unchange_name_columns = ['Status', 'Week Status', 'Week Shipment Mean', 'Week Status2']
shift_change_name_columns = ['Temp', 'Week Temp']
# env_columns1 = ['Temp-'+str(week_1), 'Week Temp-'+str(week_1)]
# env_columns2 = ['Temp-'+str(week_2), 'Week Temp-'+str(week_2)]
# env_columns3 = ['Temp-'+str(week_3), 'Week Temp-'+str(week_3)]

env_columns = ['Temp-'+str(13), 'Week Temp-'+str(13)]

cat_columns = ['Week Number']
drop_columns = ['Target', 'Week', 'Week Start', 'Week End', 'Week Status2', 'Status', 'Teisyoku', 
                'Week Teisyoku', 'Week WNDDIR','Week WNDSPD', 'Week RHUM', 'Week PRCRIN_30MIN', 
                'Week SNWFLL_30MIN', 'Week WX', 'Week Solar', 'Week Rain', 'Week Pred Temp',
                'WNDDIR', 'WNDSPD', 'RHUM', 'PRCRIN_30MIN', 'SNWFLL_30MIN', 'WX', 'Solar', 'Rain',
                'Pred Temp', 'GLBRAD', 'GLBRAD_30MIN', 'Week GLBRAD', 'Week GLBRAD_30MIN',
                'AIRTMP', 'Week AIRTMP']

In [4]:
df_week1 = df.copy()
df_week2 = df.copy()
df_week3 = df.copy()

In [5]:
df_week1 = (
        df_week1.pipe(pre.change_column_name)
        .pipe(pre.set_index_date)
        .pipe(pre.add_status)
        .pipe(pre.add_weekday)
        .pipe(pre.add_weekly_total, columns=weekly_total_columns)
        .pipe(pre.add_weekly_mean, columns=weekly_mean_columns) 
        .pipe(pre.add_week_status2)
        .pipe(pre.shift_unchange_name, shift_columns=shift_unchange_name_columns, shift_days=week_1)
        .pipe(pre.shift_change_name, shift_columns=shift_change_name_columns, shift_days=[we]ek_1)
        .pipe(pre.change_env_data, env_columns=env_columns)
        .pipe(pre.add_target, target_days=week_1) # Week Shipmentをズラす前に作成することに注意
        .pipe(pre.add_target2, target_days=week_1) # Week Shipmentをズラす前に作成することに注意
        .pipe(pre.shift_unchange_name, shift_columns=['Week Shipment'], shift_days=-7) # Week Shipmentは1週間前のデータを使用
        .pipe(pre.drop_columns, drop_columns=drop_columns)
        .pipe(pre.categorize_columns, cat_columns=cat_columns)
)

TypeError: 'int' object is not iterable

In [ ]:
df_week2 = (
        df_week2.pipe(pre.change_column_name)
        .pipe(pre.set_index_date)
        .pipe(pre.add_status)
        .pipe(pre.add_weekday)
        .pipe(pre.add_weekly_total, columns=weekly_total_columns)
        .pipe(pre.add_weekly_mean, columns=weekly_mean_columns) 
        .pipe(pre.add_week_status2)
        .pipe(pre.shift_unchange_name, shift_columns=shift_unchange_name_columns, shift_days=week_2)
        .pipe(pre.shift_change_name, shift_columns=shift_change_name_columns, shift_days=13)
        .pipe(pre.change_env_data, env_columns=env_columns)
        .pipe(pre.add_target, target_days=week_2) # Week Shipmentをズラす前に作成することに注意
        .pipe(pre.add_target2, target_days=week_2) # Week Shipmentをズラす前に作成することに注意
        .pipe(pre.shift_unchange_name, shift_columns=['Week Shipment'], shift_days=-7) # Week Shipmentは1週間前のデータを使用
        .pipe(pre.drop_columns, drop_columns=drop_columns)
        .pipe(pre.categorize_columns, cat_columns=cat_columns)
)

In [ ]:
df_week3 = (
        df_week3.pipe(pre.change_column_name)
        .pipe(pre.set_index_date)
        .pipe(pre.add_status)
        .pipe(pre.add_weekday)
        .pipe(pre.add_weekly_total, columns=weekly_total_columns)
        .pipe(pre.add_weekly_mean, columns=weekly_mean_columns) 
        .pipe(pre.add_week_status2)
        .pipe(pre.shift_unchange_name, shift_columns=shift_unchange_name_columns, shift_days=week_3)
        .pipe(pre.shift_change_name, shift_columns=shift_change_name_columns, shift_days=13)
        .pipe(pre.change_env_data, env_columns=env_columns)
        .pipe(pre.add_target, target_days=week_3) # Week Shipmentをズラす前に作成することに注意
        .pipe(pre.add_target2, target_days=week_3) # Week Shipmentをズラす前に作成することに注意
        .pipe(pre.shift_unchange_name, shift_columns=['Week Shipment'], shift_days=-7) # Week Shipmentは1週間前のデータを使用
        .pipe(pre.drop_columns, drop_columns=drop_columns)
        .pipe(pre.categorize_columns, cat_columns=cat_columns)
)

In [ ]:
# 学習に使用される特徴量の確認
df_week2.columns

Index(['Week Temp', 'Week High Temp', 'Week Low Temp', 'Week Shipment',
       'Shipment', 'Temp', 'High Temp', 'Low Temp', 'Week Status',
       'Week Shipment Mean', 'Temp-13', 'Week Temp-13', 'Target2',
       'Week Number_1', 'Week Number_2', 'Week Number_3', 'Week Number_4',
       'Week Number_5', 'Week Number_6', 'Week Number_7', 'Week Number_8',
       'Week Number_9', 'Week Number_10', 'Week Number_11', 'Week Number_12',
       'Week Number_13', 'Week Number_14', 'Week Number_15', 'Week Number_16',
       'Week Number_17', 'Week Number_18', 'Week Number_19', 'Week Number_20',
       'Week Number_21', 'Week Number_22', 'Week Number_23', 'Week Number_24',
       'Week Number_25', 'Week Number_26', 'Week Number_27', 'Week Number_28',
       'Week Number_29', 'Week Number_30', 'Week Number_31', 'Week Number_32',
       'Week Number_33', 'Week Number_34', 'Week Number_35', 'Week Number_36',
       'Week Number_37', 'Week Number_38', 'Week Number_39', 'Week Number_40',
       'Week 

In [ ]:
# データ範囲を指定
X = df_week2['2018-08-06':].drop('Target2', axis=1)
y = df_week2['2018-08-06':]['Target2']

In [ ]:
X.shape, y.shape

((2136, 65), (2136,))

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = ut.train_val_test_split2(X, y, '2022-08-06', '2024-4-01')

In [ ]:
X_train.shape, y_train.shape

((1462, 65), (1462,))

In [ ]:
# スケーラーを初期化
# scaler_X = StandardScaler()
# scaler_y = StandardScaler()
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# トレーニングデータのみに対してフィッティング
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

X_val_scaled = scaler_X.transform(X_val)
y_val_scaled = scaler_y.fit_transform(y_val.values.reshape(-1, 1))

# テストデータをトレーニングデータのスケールに変換
X_test_scaled = scaler_X.transform(X_test)

In [ ]:
# 欠損値がある場合は補間
if np.isnan(X_train_scaled).sum() > 0:
    # 平均値で置換
    imputer = SimpleImputer(strategy='mean')
    X_train_scaled = imputer.fit_transform(X_train_scaled)

if np.isnan(X_val_scaled).sum() > 0:
    # 平均値で置換
    imputer = SimpleImputer(strategy='mean')
    X_val_scaled = imputer.fit_transform(X_val_scaled)

if np.isnan(X_test_scaled).sum() > 0:
    # 平均値で置換
    imputer = SimpleImputer(strategy='mean')
    X_test_scaled = imputer.fit_transform(X_test_scaled)

In [ ]:
X_train_scaled.shape, y_train_scaled.shape

((1462, 65), (1462, 1))

In [ ]:
# モデルの入力に合わせてデータを整形
# X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
# X_val_scaled = X_val_scaled.reshape((X_val_scaled.shape[0], X_val_scaled.shape[1], 1))
# X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_val_scaled = X_val_scaled.reshape((X_val_scaled.shape[0], 1, X_val_scaled.shape[1]))
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

In [ ]:
# データセットのサイズ確認
len(X_train), len(X_val), len(X_test)

(1462, 605, 71)

In [ ]:
# データセットの形状確認
X_train_scaled.shape, y_train_scaled.shape, X_val_scaled.shape, y_val_scaled.shape, X_test_scaled.shape

((1462, 1, 65), (1462, 1), (605, 1, 65), (605, 1), (71, 1, 65))

In [ ]:
# モデルパラメータの設定
# param_grid = {
#     'dense_units': [32, 64, 128, 256, 512, 1028],
#     'gru_units': [32, 64, 128, 256, 512, 1028],
#     'learning_rate': [0.001],
#     'batch_size': [32]
# }
param_grid = {
    'dense_units': [32],
    'gru_units': [32],
    'learning_rate': [0.001],
    'batch_size': [32]
}

import itertools

param_combinations = list(itertools.product(
    param_grid['dense_units'],
    param_grid['gru_units'],
    param_grid['learning_rate'],
    param_grid['batch_size']
))

param_list = []
for dense_units, gru_units, learning_rate, batch_size in param_combinations:
    params = {
        'dense_units': dense_units,
        'gru_units': gru_units,
        'learning_rate': learning_rate,
        'batch_size': batch_size
    }
    param_list.append(params)

In [ ]:
# 保存先を指定
tracking_uri = '/home/code/mlflow/mlruns'
mlflow.set_tracking_uri(tracking_uri)

experiment_name = "gru_experiment11"
mlflow.set_experiment(experiment_name)

# experiment_idを取得
experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

# mlflowのシステムメトリクスの記録設定
os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "true"

In [ ]:
# モデルパラメータ
for params in param_list:
    dense_units = params['dense_units']
    gru_units = params['gru_units']
    learning_rate = params['learning_rate']
    batch_size = params['batch_size']

    # モデルの構築
    model = gru.GRU_single(input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]), 
                          dense_units=dense_units, 
                          gru_units=gru_units, 
                          lr=learning_rate
                        )
    # モデルの視覚化
    plot_model(model,show_shapes=True) 
    
    # EarlyStoppingの設定
    early_stopping = EarlyStopping(monitor='val_loss', patience=40, restore_best_weights=True)

    with mlflow.start_run(experiment_id=experiment_id):
        # モデルの訓練
        history = model.fit(X_train_scaled, y_train_scaled, epochs=10000, batch_size=batch_size, validation_data=(X_val_scaled, y_val_scaled), callbacks=[early_stopping], verbose=2)
        
        # 検証データでの損失を評価
        val_loss = model.evaluate(X_val_scaled, y_val_scaled)
        print(f'val_loss: {val_loss}')

        # モデルの保存
        model.save('/home/code/mlflow/artifacts/models/keras_model.h5')

        # モデルのパラメータを記録
        mlflow.log_param('dense_units', dense_units)
        mlflow.log_param('gru_units', gru_units)
        # mlflow.log_param('learning_rate', learning_rate)
        mlflow.log_param('batch_size', batch_size)

        # 損失を記録
        mlflow.log_metric('last_val_loss', val_loss)

        for epoch in range(len(history.history['loss'])):
            mlflow.log_metric('train_loss', history.history['loss'][epoch], step=epoch)
            mlflow.log_metric('val_loss', history.history['val_loss'][epoch], step=epoch)

        # X_val, X_testに対する予測結果をcsvで保存
        df_val = X_val.copy()
        y_val_pred = model.predict(X_val_scaled)
        y_val_pred = scaler_y.inverse_transform(y_val_pred)
        y_val_, y_val_pred_ = ut.Target2_transform(X_val, y_val, y_val_pred)
        df_val['Ans'] = y_val_
        df_val['Pred'] = y_val_pred_
        
        df_test = X_test.copy()
        y_test_pred = model.predict(X_test_scaled)
        y_test_pred = scaler_y.inverse_transform(y_test_pred)
        y_test_, y_test_pred_ = ut.Target2_transform(X_test, y_test, y_test_pred)
        df_test['Ans'] = y_test_
        df_test['Pred'] = y_test_pred_

        # df_valにおいて、誤差と絶対誤差を計算
        df_val['Error'] = df_val['Ans'] - df_val['Pred']
        df_val['Abs Error'] = abs(df_val['Error'])

        # 最大誤差、最小誤差、平均誤差を計算（Abs Errorが0の行を除いて計算）
        val_max_error = df_val[df_val['Abs Error'] != 0]['Abs Error'].max()
        val_min_error = df_val[df_val['Abs Error'] != 0]['Abs Error'].min()
        val_mean_abs_error = df_val[df_val['Abs Error'] != 0]['Abs Error'].mean()
        mlflow.log_metric('val_max_error', val_max_error)
        mlflow.log_metric('val_min_error', val_min_error)
        mlflow.log_metric('val_mean_abs_error', val_mean_abs_error)

        df_val.to_csv('/home/code/mlflow/artifacts/csv/val_pred.csv')
        mlflow.log_artifact('/home/code/mlflow/artifacts/csv/val_pred.csv')
        df_test.to_csv('/home/code/mlflow/artifacts/csv/test_pred.csv')
        mlflow.log_artifact('/home/code/mlflow/artifacts/csv/test_pred.csv')

        plt.figure(figsize=(10, 5))
        plt.plot(df_val['Ans'], label='Actual')
        plt.plot(df_val['Pred'], label='Predict')
        plt.legend()
        plt.savefig('/home/code/mlflow/artifacts/images/val_pred.png')
        mlflow.log_artifact('/home/code/mlflow/artifacts/images/val_pred.png')
        plt.close()

        # モデルの保存
        mlflow.log_artifact('/home/code/mlflow/artifacts/models/keras_model.h5')

2024-06-11 12:02:36.585989: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-11 12:02:36.641047: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-11 12:02:36.641681: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-11 12:02:36.643145: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


2024/06/11 12:02:39 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2024/06/11 12:02:40 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics becau

Epoch 1/10000


2024-06-11 12:02:44.787016: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-11 12:02:47.280985: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


46/46 - 17s - loss: 0.0577 - val_loss: 0.0175 - 17s/epoch - 363ms/step
Epoch 2/10000
46/46 - 2s - loss: 0.0093 - val_loss: 0.0121 - 2s/epoch - 47ms/step
Epoch 3/10000
46/46 - 2s - loss: 0.0052 - val_loss: 0.0112 - 2s/epoch - 52ms/step
Epoch 4/10000
46/46 - 2s - loss: 0.0044 - val_loss: 0.0108 - 2s/epoch - 46ms/step
Epoch 5/10000
46/46 - 2s - loss: 0.0042 - val_loss: 0.0103 - 2s/epoch - 46ms/step
Epoch 6/10000
46/46 - 2s - loss: 0.0042 - val_loss: 0.0100 - 2s/epoch - 39ms/step
Epoch 7/10000
46/46 - 2s - loss: 0.0041 - val_loss: 0.0102 - 2s/epoch - 35ms/step
Epoch 8/10000
46/46 - 2s - loss: 0.0040 - val_loss: 0.0097 - 2s/epoch - 45ms/step
Epoch 9/10000
46/46 - 2s - loss: 0.0041 - val_loss: 0.0101 - 2s/epoch - 39ms/step
Epoch 10/10000
46/46 - 2s - loss: 0.0040 - val_loss: 0.0099 - 2s/epoch - 39ms/step
Epoch 11/10000
46/46 - 1s - loss: 0.0040 - val_loss: 0.0102 - 1s/epoch - 28ms/step
Epoch 12/10000
46/46 - 2s - loss: 0.0040 - val_loss: 0.0094 - 2s/epoch - 34ms/step
Epoch 13/10000
46/46 - 1

2024/06/11 12:03:52 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/06/11 12:03:52 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
